<a href="https://colab.research.google.com/github/sumanth4341/Link-Prediction-on-FB-Pages/blob/main/link_prediction_fb_pages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Link Prediction 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sknetwork 
import networkx as nx
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm 
import random
from node2vec import Node2Vec

In [ ]:
# Load edges details
with open('fb_') as f: 
  fb_links = f.read().splitlines()


print(len(fb_links))

88234


In [ ]:
# Take nodes in two seperate lists
node_list1 = []
node_list2 = []

for i in tqdm(fb_links): 
  node_list1.append(i.split(' ')[0])
  node_list2.append(i.split(' ')[1])


fb_df = pd.DataFrame({'node_1':node_list1,
                      'node_2':node_list2})
fb_df.head()

100%|██████████| 88234/88234 [00:00<00:00, 215908.05it/s]


,node_1,node_2
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5


In [ ]:
# Create Graph 
G = nx.from_pandas_edgelist(fb_df,'node_1','node_2',create_using=nx.Graph())


In [ ]:
# combine all the nodes in the list
node_list = node_list1 + node_list2

node_list = list(dict.fromkeys(node_list))

# Build Adjacency matrix
adj_G = nx.to_numpy_matrix(G,node_list)
adj_G.shape

(4039, 4039)

In [ ]:
# get unconnected node-pairs
all_unconnected_pairs = []

# traverse adjacency matrix 
offset=0
for i in tqdm(range(adj_G.shape[0])): 
  for j in range(offset,adj_G.shape[1]): 
    if i!=j :
      if nx.shortest_path_length(G,str(i),str(j)) <=2:
        if adj_G[i,j] == 0: 
          all_unconnected_pairs.append([node_list[i],node_list[j]])
  offset+=1
len(all_unconnected_pairs)

  1%|          | 25/4039 [00:22<1:00:24,  1.11it/s]


KeyboardInterrupt: ignored

In [ ]:
node_1_unlinked = [i[0] for i in all_unconnected_pairs]
node_2_unlinked = [i[1] for i in all_unconnected_pairs]
data = pd.DataFrame({'node_1': node_1_unlinked,
                     'node_2': node_2_unlinked})

# add target variable 'link'
data['link']=0
data

,node_1,node_2,link
0,0,22,0
1,0,526,0
2,0,36,0
3,0,54,0
4,0,56,0
...,...,...,...
19013,606,586,0
19014,606,541,0
19015,592,573,0
19016,592,541,0


In [ ]:
initial_node_count = len(G.nodes)

fb_df_temp = fb_df.copy()

# empty list to store removable links
omissible_links_index = []

for i in tqdm(fb_df.index.values):

  # remove a node pair and create a new graph
  G_temp = nx.from_pandas_edgelist(fb_df_temp.drop(index=i),'node_1','node_2',create_using=nx.Graph())

  # check there is no splitting of graph and number of nodes is same
  if(nx.number_connected_components(G_temp)==1) and (len(G_temp.nodes)==initial_node_count):
    omissible_links_index.append(i)
    fb_df_temp = fb_df_temp.drop(index=i)

len(omissible_links_index)

100%|██████████| 2102/2102 [00:12<00:00, 167.62it/s]


1483

In [ ]:
# create dataframe of removable edges
fb_df_ghost = fb_df.loc[omissible_links_index]

# Add the target variable link 
fb_df_ghost['link']=1

data = data.append(fb_df_ghost[['node_1','node_2','link']],ignore_index=True)
data['link'].value_counts()

0    19018
1     1483
Name: link, dtype: int64

In [ ]:
# drop removable edges
fb_df_partial = fb_df.drop(index=fb_df_ghost.index.values)

# build graph 
G_data = nx.from_pandas_edgelist(fb_df_partial,"node_1","node_2",create_using=nx.Graph())

In [ ]:
# Generate Walks
node2vec = Node2Vec(G_data,walk_length = 16,num_walks=50,dimensions=100)
n2w_model = node2vec.fit(window=7,min_count=1)

Computing transition probabilities:   0%|          | 0/620 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 50/50 [00:18<00:00,  2.64it/s]


In [ ]:
x = [(n2w_model[str(i)])+(n2w_model[str(j)]) for i,j in zip(data['node_1'],data['node_2'])]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(np.array(x),data['link'],test_size=0.2,random_state=42)

In [ ]:
lr = LogisticRegression(class_weight="balanced")

In [ ]:
lr.fit(xtrain,ytrain)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(class_weight='balanced')

In [ ]:
predictions = lr.predict_proba(xtest)

In [ ]:
roc_auc_score(ytest,predictions[:,1])

0.8159662060075128